To demonstrate the Spark Streaming API in action, we will use a simple wordCount example from the Spark documentation. To get this working, first open a terminal window, or a terminal tab on JupyterHub, and run the following command:

<code>ncat -lk 9999</code>

Now, anything you type into that window will be visible at port 9999. The code block below sets up a spark Streaming Context (convention is to name it <code>ssc</code>) and tells spark to listen for anything that pops up on port 9999.

We will pick up what we'll type on the other window and will run our familiar WordCount steps to... count the words we just typed.

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
#sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream("localhost", 9999)

words = lines.flatMap(lambda line : line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCount = pairs.reduceByKey(lambda a, b: a + b)

wordCount.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()

The methods available in the Spark Streaming API are very similar to the ones from the RDD API. The example below sets up a Spark Streaming Context and tells Spark to **poll** (or monitor) a specific directory in our filesystem. Whenever a new file gets moved into that directory, Spark will ingest its contents as Text, just like we did with the RDD API. 

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

#sc = pyspark.SparkContext()
ssc = StreamingContext(sc,10)

records = ssc.textFileStream("/home/user74/scratch/")

rows = records.map(lambda line : line.split(","))
rows.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()

As we've seen before, the RDD API is great, but the SparkSQL API really makes manipulating data feel more familiar for those who already know SQL or use R or Pandas regularly. You can leverage the SparkSQL API by making Spark Streaming populate a DataFrame as it reads the stream: 

In [1]:
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext()

spark = SQLContext(sc)

my_dataframe = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

This tells spark to listen on port 9999, just like we did before with SparkStreamingContext. The difference here is that we are telling Spark we want the contents of the stream to go into a DataFrame! Next, we inform Spark of what operations we wish to perform on our Stream. 

In this example, we will simply write the contents of the Stream to "memory" and call it "new_dataframe". We will also tell Spark to "append" new records to "new_dataframe" as they arrive. This is equivalent to registering a temp table like we did before, then populating it with the incoming contents of the stream!

In [3]:
my_dataframe.writeStream.format("memory").queryName("new_dataframe").outputMode("append").start()

You can verify that indeed we now have a registered table called "new_dataframe":

In [5]:
spark.tableNames()

['new_dataframe']

And you can use SQL to query this table. Here, we assume the contents of the stream are comma separated values and we split them into columns:

In [25]:
spark.sql("SELECT SPLIT(value,',')[0] AS Col1, SPLIT(value,',')[1] as Col2 FROM new_dataframe").collect()

[Row(Col1='this is a test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='this', Col2=' is', Col3=' a')]

Here we have demonstrated how Spark Streaming works by simulating a message broker that passes Text content over to Spark. This is a fairly common use case, but you will also often come across messages that are in a specific format, like JSON for example.

The flow to set up a stream that handles files of a specific format is very similar to the examples above, it suffices to change the "format" option! See here for other supported formats: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#api-using-datasets-and-dataframes

## Exercise 1 - What is the Spread?

In finance, the term **spread** is often used to refer to the difference between two metrics of interest. In the stock market in particular, the **Bid-Ask spread** is one of the many tools used to help inform decision making around a trader's positions in the market. The Bid-Ask spread is simply the difference between the price traders selling a security are asking, **the ask**, and the price traders buying that security are offering to pay, **the bid**. 

A prized piece of information that many firms pay good money to acquire is known as a **BBO** - the **Best Bid-Offer** datset. BBO datasets are simple tabular-formatted collections of data with 3 key columns: a timestamp, the best bid and the best offer for a given security at that exact timestamp. Stock brokers have a fiduciary duty to get the best bid for their clients selling a security and the best offer for their clients buying a security. Hence the usefulness of BBO data... but that is not all! If you have current BBO data, you can also keep an eye on the **spread** on a given security in real-time. Knowing this quantity in real-time can be used in many ways, the simplest of which is as a gauge of the **liquidity** of a security. In general, if the **spread** on a security is small, that suggests there is a hot market for that security (i.e. people are actively buying and selling the seurity). Conversely, if the spread is large, that suggests the market is not really interested in trading that security at that time.

We will not show you how to create your own BBO dataset today, but we will use one to keep an eye on the spread of a certain stock. 

First, let's simulate a real-time feed from a BBO provider. To do this, run the following command on a terminal window:

<code>stdbuf -oL cat data/14081.csv | ncat localhost 9999</code>

Now, use the Structured Streaming approach we've seen before to read the BBO data into a DataFrame and get the **Spread** second-by-second. If you are looking for a challenge, try computing the **average spread** on a minute-by-minute basis.

## Exercise 2 - Anomaly Detection

Now let's look at another application of Spark: anomaly detection. In this problem, a fictional Utility Company started using Machine Learning to determine the price it charges its customers depending on the hour of the day. You believe this move on the part of the Utility Company causing you to pay way too much for electricity and have decided to put together a dossier exposing how their algorithm is out of whack. One way of exposing the weakness of their algorithm would be to catch anomalies in the price they are charging. The company themselves defines an anomaly as "**a 2 standard deviation or larger increase over the average price for the same hour over the past two weeks.**"

Use the SparkSQL API to read the <code>utility.csv</code> and find instances of anomalous pricing.